# Разработка веб-приложения с LLM

GigaChat API

Создание аккаунта: https://developers.sber.ru/studio/login

Авторизация: https://developers.sber.ru/docs/ru/gigachat/individuals-quickstart

ngrok

Создание аккаунта ngrok: https://dashboard.ngrok.com/

Настройка доступа: https://dashboard.ngrok.com/get-started/setup/windows

## Использование GigaChat с Langchain

In [ ]:
import os
from google.colab import userdata

In [ ]:
auth_key = userdata.get('giga_api')
os.environ["GIGACHAT_CREDENTIALS"] = auth_key

In [ ]:
import json
import requests

url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

payload = {
    'scope': 'GIGACHAT_API_PERS'
}

headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': 'application/json',
    'RqUID': '09cde699-e915-475b-b3af-b737fc80a7a9',
    'Authorization': f'Basic {os.environ["GIGACHAT_CREDENTIALS"]}'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

access_token = json.loads(response.text)["access_token"]

/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


## Использование GigaChat через Langchain

In [ ]:
%pip install --upgrade --quiet langchain-gigachat

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 4.6 MB/s eta 0:00:00


In [ ]:
from langchain_gigachat import GigaChat

chat = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

In [ ]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage( # системный промпт
        content="Ты ИИ ассистент. Аргументированно отвечай на любые вопросы."
    ),
    HumanMessage(content='Как вырастить "единорога"?'),
]

print(chat.invoke(messages).content) # invoke() — отправляет запрос в LLM

Термин «единорог» в контексте бизнеса используется для обозначения частной компании, оценочная стоимость которой превышает один миллиард долларов. Для того чтобы компания могла стать единорогом, необходимо учитывать несколько ключевых факторов:

1. **Уникальная бизнес-модель**: Компании-единороги обычно обладают инновационной и уникальной бизнес-моделью, которая решает конкретную проблему или удовлетворяет спрос, ранее не замеченный рынком.
 шляпах стартапов.

2. **Качественный продукт или услуга**: Продукт должен быть востребованным, удобным и конкурентоспособным на рынке. В идеале он должен иметь высокий потенциал роста и масштабируемость.

3. **Персонализированный подход к маркетингу и продажам**: Эффективная стратегия продвижения продукта и привлечение клиентов — важный аспект успеха стартапа. Персонализация маркетинга помогает быстрее завоевать доверие целевой аудитории.

4. **Сильная команда управления**: Успех любой компании во многом зависит от качества её руководства и команды

## Создание веб-приложения

In [ ]:
%pip install -q streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 85.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 120.4 MB/s eta 0:00:00


### Разработка веб-приложения с GigaChat и Streamlit

Код вставить в "app.py"

In [ ]:
import os
import json
import requests
import streamlit as st
from langchain_gigachat import GigaChat
from langchain_core.messages import HumanMessage, SystemMessage

# Пример предшествующего промпта и роли. Измените это на свои значения.
PRE_PROMPT = "Ты выступаешь в роли ИИ ассистента. Подробно отвечай на вопросы, чтобы понятно было даже школьнику. Избегай опасных тем ответом: 'Тебе ещё рано об этом знать!'" # базовый промпт
ROLE = 'Ассистент'  # Измените это на свою роль

MODELS = { # выбор моделей
    'GigaChat Light': 'GigaChat Light',
    'GigaChat Pro': 'GigaChat Pro',
    'GigaChat Max': 'GigaChat Max',
}

# авторизация GigaChat
url = "https://ngw.devices.sberbank.ru:9443/api/v2/oauth"

payload = {
    'scope': 'GIGACHAT_API_PERS'
}

headers = {
    'Content-Type': 'application/x-www-form-urlencoded',
    'Accept': 'application/json',
    'RqUID': '52f2fec6-f9e1-4334-b08f-69716f7db05e',
    'Authorization': f'Basic {os.environ["GIGACHAT_CREDENTIALS"]}'
}

response = requests.request("POST", url, headers=headers, data=payload, verify=False)

access_token = json.loads(response.text)["access_token"]

chat = GigaChat(verify_ssl_certs=False, scope="GIGACHAT_API_PERS")

# Установка начальной конфигурации страницы
st.set_page_config(
    page_title="GigaApp",         # Устанавливает заголовок страницы.
    page_icon=":robot_face:",     # Устанавливает иконку страницы.
    layout="wide"                 # Устанавливает макет страницы — широкий формат
)

def setup_session_state():
    """Настройка начального состояния сессии"""
    if 'chat_dialogue' not in st.session_state:
        st.session_state['chat_dialogue'] = []
    if 'model' not in st.session_state:
        st.session_state['model'] = 'GigaChat Light'
    if 'llm' not in st.session_state:
        st.session_state['llm'] = MODELS[st.session_state['model']]
    if 'role' not in st.session_state:
        st.session_state['role'] = ROLE
    if 'prompt' not in st.session_state:
        st.session_state['prompt'] = PRE_PROMPT

def render_settings():
    """Отображение настроек на основной странице"""
    st.header("Настройки бота GigaChat")
    st.session_state['role'] = st.text_input('Введите роль бота:', st.session_state['role'])
    st.session_state['prompt'] = st.text_area('Введите промпт:', st.session_state['prompt'])
    selected_model = st.selectbox('Выберите модель:', list(MODELS.keys()))
    st.session_state['llm'] = MODELS[selected_model]

def render_chat_history():
    """Отображение истории чата"""
    for message in st.session_state['chat_dialogue']:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

def handle_user_input():
    """Обработка ввода пользователя"""
    user_input = st.chat_input("Введите ваш вопрос:")
    if user_input:
        st.session_state['chat_dialogue'].append({
            "role": st.session_state['role'],
            "content": user_input
        })
        with st.chat_message(st.session_state['role']):
            st.markdown(user_input)
        generate_assistant_response(user_input)

def generate_assistant_response(user_input):
    """Генерация ответа ассистента"""
    # Собираем весь диалог в одну строку
    dialogue = "\n".join([f"{msg['role'].capitalize()}: {msg['content']}" for msg in st.session_state['chat_dialogue']])
    dialogue += "\nAssistant: "

    messages = [
        SystemMessage(content=st.session_state['prompt']),
        HumanMessage(content=dialogue),
    ]

    response = chat.invoke(messages).content

    # Отображаем и сохраняем ответ
    full_response = ''.join(response)
    st.session_state['chat_dialogue'].append({"role": "assistant", "content": full_response})
    with st.chat_message("assistant"):
        st.markdown(full_response)

# Основная функция приложения
def main():
    """Основная функция приложения"""
    setup_session_state()     # настройка состояния сессии
    render_settings()         # рендеринг боковой панели
    render_chat_history()     # история чата
    handle_user_input()       # пользовательский ввод

if __name__ == "__main__":
    main()

In [ ]:
!echo > app.py

In [ ]:
from google.colab import files
from google.colab import drive

files.view("app.py")

<IPython.core.display.Javascript object>

### Деплой приложения с помощью ngrok

In [ ]:
!pip install pyngrok

In [ ]:
from threading import Thread
from pyngrok import ngrok

In [ ]:
ngrok_key = userdata.get('ngrok_key')
ngrok.set_auth_token(ngrok_key)

In [ ]:
def run_streamlit():
    os.system('streamlit run /content/app.py --server.port 8501')

In [ ]:
thread = Thread(target=run_streamlit)
thread.start()

In [ ]:
public_url = ngrok.connect(addr='8501', proto='http', bind_tls=True)
print('Ваше приложение Streamlit доступно по адресу:', public_url)

Ваше приложение Streamlit доступно по адресу: NgrokTunnel: "https://ba61dce225bb.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
ngrok.kill()